# Example Application of the Filesystem Implementation
This notebook outlines how `hards` can be used to store and access data for scientific experiments.

`hards` itself has no external dependencies and is written using Python standard libraries such as `pathlib` and `json`. However, this example uses a number of a visualisation and scientific computing libraries to provide a more coherent and useful example. It is therefore required that this notebook is run in an environment with the development dependencies installed as detailed in the README.


The approximation of $\pi$ using [Monte Carlo methods](https://en.wikipedia.org/wiki/Monte_Carlo_method#Overview) is a well-know introductory problem to scientific computing and Monte Carlo methods. In this notebook, we will compare Monte Carlo sampling versus quasi-Monte Carlo sampling and conclude which is better at approximating $\pi$. The basic principle is to generate $N$ points on the unit square $[0, 1]^2$ and check how many, $p$, are within the unit circle (centre $(0, 0)$ and radius $1$). 

$$
\pi\approx\frac{4p}{N}
$$

## Setup Helper Functions

First, we will setup a number of helper functions to sample data on the unit square and check whether the sample is within the unit circle. The data will be a list of dictionaries, one for each sample point.

In [ ]:
import random
from typing import Literal

from scipy.spatial.distance import euclidean
from scipy.stats import qmc


def sample_unit_square(num: int, method: Literal["sobol", "random"]) -> list:
    """Sample `num` points from the unit square."""
    # Generate required number of 2D samples

    if method == "sobol":
        sampler = qmc.Sobol(2)
        samples = sampler.random(num).tolist()
    else:
        samples = []
        for _ in range(num):
            samples.append([random.uniform(0.0, 1.0), random.uniform(0.0, 1.0)])

    data = []
    for x, y in samples:
        # Calculate the Euclidean distance of the sample from the cirlce centre
        radius = euclidean([x, y], [0.0, 0.0])

        # Check whether the point are within the unit circle
        in_circle = radius <= 1.0

        data.append({"x": x, "y": y, "r": radius, "in_circle": in_circle})

    return data

The next helper functions will visualise the samples and calculate an approximate of $\pi$.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


def plot_samples(samples: list) -> None:
    """Plot samples on the unit circle."""
    samples_df = pd.DataFrame(samples)

    ax = sns.scatterplot(
        samples_df,
        x="x",
        y="y",
        hue="in_circle",
        style="in_circle",
    )

    ax.add_patch(plt.Circle((0, 0), 1.0, edgecolor="black", facecolor="None"))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    sns.move_legend(ax, loc="upper left")

    return ax.figure


def approximate_pi(samples: list) -> float:
    """Approximate pi from the samples."""
    samples_df = pd.DataFrame(samples)

    return (4.0 * len(samples_df[samples_df["in_circle"]])) / len(samples_df)

Finally, this function converts the list of samples into a dictionary indexed by the string `'sample<id>'`.

In [ ]:
def samples_to_dict(samples: list) -> dict:
    """Convert the samples into a dictionary."""
    return {f"sample{i}": v for i, v in enumerate(samples)}

## Create the HARDS Databse
The `FilesystemDatabase` will be created inside of a temporary directory meaning all data from the example will be cleaned up upon exiting the notebook (the OS should handle this, the end of the notebook will also do this explicitly).

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

from hards.filesystem import FilesystemDatabase

temp_directory = TemporaryDirectory()
database = FilesystemDatabase.create_database(Path(temp_directory.name) / "database")

## Initial Random Sampling
Some initial sampling will help verify the helper functions work and illustrate basic `hards` dataset management. 

First, lets create the dataset where we will store these initial random samples.

In [ ]:
initial_random_dataset = database.create_dataset("initial_random")

We might want to include some metadata on this dataset, such as what method was used to create the samples.

In [ ]:
initial_random_dataset.add_data({"method": "random"})

Next, we can create `100` samples using Monte Carlo (random) sampling, visualise how they are distributed in the unit square, and calculate an approximation of $\pi$.

In [ ]:
initial_random_samples = sample_unit_square(100, "random")

print(f"pi approximation: {approximate_pi(initial_random_samples)}")

fig = plot_samples(initial_random_samples)
fig.show()

We can add each of the samples as a 'datapoint' which contains some data.

In [ ]:
for i, sample in enumerate(initial_random_samples):
    datapoint = initial_random_dataset.create_datapoint(f"{i}")
    datapoint.add_data(sample)

We can then reload the data from `hards` and verify it produces an identical visualisation and $\pi$ approximation

In [ ]:
initial_random_samples = [
    dp.data for dp in initial_random_dataset.recursively_get_datapoints()
]

print(f"pi approximation: {approximate_pi(initial_random_samples)}")

fig = plot_samples(initial_random_samples)
fig.show()

## Full Random Sampling
Now that the basics of creating a `hards` database and dataset has been demonstrated, it is time to create a full dataset of $8092$ samples to get a more accurate estimation of $\pi$.

We have an existing dataset of $100$ samples, and it would be a shame to waste them. Therefore, we can extend the existing dataset and add just $8092$ new samples.

In [ ]:
# create a new dataset 'under' the full dataset
full_random_dataset = initial_random_dataset.create_dataset("full_random")

# create more samples and add it to this new dataset
for i, sample in enumerate(sample_unit_square(8092, "random")):
    datapoint = full_random_dataset.create_datapoint(f"{i}")
    datapoint.add_data(sample)

This new dataset (`full_random_dataset`) is a child of the `initial_random` dataset. In `hards`, children provide access to their own data _and_ their parents data. Therefore, we would expect `full_random_dataset` has $8192$ datapoints despite us only adding $8092$.

In [ ]:
print(len(full_random_dataset.recursively_get_datapoints()))

## Full Sobol Sampling

We can repeat this with Sobol sampling so we have two datasets of the same size to compare against. We will do all of the samples into one dataset rather than messing around with sub-datasets.

In [ ]:
full_sobol_dataset = database.create_dataset("full_sobol")

for i, sample in enumerate(sample_unit_square(8192, "sobol")):
    datapoint = full_sobol_dataset.create_datapoint(f"{i}")
    datapoint.add_data(sample)

## Conclusion

Lets (for examples sake) load everything from scratch before performing some analysis on our results.

In [ ]:
database = FilesystemDatabase(Path(temp_directory.name) / "database")

random_dataset = database.recursively_get_dataset("initial_random/full_random")
sobol_dataset = database.get_dataset("full_sobol")

random_datapoints = [dp.data for dp in random_dataset.recursively_get_datapoints()]
sobol_datapoints = [dp.data for dp in sobol_dataset.recursively_get_datapoints()]

Finally, lets see which method performs the best by checking their errors.

Normally (but not always) the Sobol sampling will perform the best.

In [ ]:
from math import pi

print(f"Random error: {abs(pi - approximate_pi(random_datapoints))}")
print(f"Sobol error : {abs(pi - approximate_pi(sobol_datapoints))}")

temp_directory.cleanup()